In [1]:
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch
import tensorflow as tf

In [2]:
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/iris.csv",na_values=['NA', '?'])

In [3]:
# Convert to numpy - Classification
x = df[['sepal_l', 'sepal_w', 'petal_l', 'petal_w']].values
dummies = pd.get_dummies(df['species']) # Classification
species = dummies.columns
y = dummies.values

print(y.shape,x.shape)

(150, 3) (150, 4)


In [5]:
# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [6]:
# Build neural network
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(y.shape[1],activation='softmax')) # Output
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Train on 112 samples, validate on 38 samples
Epoch 1/1000
112/112 - 1s - loss: 1.0970 - accuracy: 0.3393 - val_loss: 1.0452 - val_accuracy: 0.3158
Epoch 2/1000
112/112 - 0s - loss: 0.9997 - accuracy: 0.3750 - val_loss: 0.9687 - val_accuracy: 0.4211
Epoch 3/1000
112/112 - 0s - loss: 0.9285 - accuracy: 0.3839 - val_loss: 0.9133 - val_accuracy: 0.3158
Epoch 4/1000
112/112 - 0s - loss: 0.8866 - accuracy: 0.3393 - val_loss: 0.8796 - val_accuracy: 0.3158
Epoch 5/1000
112/112 - 0s - loss: 0.8591 - accuracy: 0.4375 - val_loss: 0.8482 - val_accuracy: 0.6579
Epoch 6/1000
112/112 - 0s - loss: 0.8351 - accuracy: 0.6607 - val_loss: 0.8204 - val_accuracy: 0.7895
Epoch 7/1000
112/112 - 0s - loss: 0.8115 - accuracy: 0.7232 - val_loss: 0.7944 - val_accuracy: 0.6842
Epoch 8/1000
112/112 - 0s - loss: 0.7870 - accuracy: 0.7946 - val_loss: 0.7742 - val_accuracy: 0.8947
Epoch 9/1000
112/112 - 0s - loss: 0.7684 - accuracy: 0.8661 - val_loss: 0.7550 - val_accuracy: 0.8684
Epoch 10/1000
112/112 - 0s - loss: 0.

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 50)                250       
_________________________________________________________________
dense_4 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 78        
Total params: 1,603
Trainable params: 1,603
Non-trainable params: 0
_________________________________________________________________


In [9]:
from sklearn.metrics import accuracy_score

pred = model.predict(x_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Accuracy: {correct}")

Accuracy: 1.0


In [10]:
#Testing the model on test set
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

38/38 [==============================] - ETA: 0s - loss: 0.0297 - accuracy: 1.00 - 0s 262us/sample - loss: 0.0424 - accuracy: 1.0000
Test loss: 0.04242380669242457
Test accuracy: 1.0


<hr><h3>I'm gonna tune using keras-tuner and see if i can do better</h3>

In [14]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units',min_value=50,
                                         max_value=80,
                                         step=10),
                                         input_dim=x.shape[1],
                                         activation='relu'))
    model.add(Dense(units=hp.Int('units',min_value=20,
                                         max_value=50,
                                         step=10),
                                         activation='relu'))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])
    return model

In [15]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='iris')

In [16]:
tuner.search(x_train, y_train,epochs=20,
             validation_data=(x_train, y_train))

Train on 112 samples, validate on 112 samples
Epoch 1/20
112/112 [==============================] - ETA: 0s - loss: 1.3775 - accuracy: 0.25 - 1s 5ms/sample - loss: 1.1136 - accuracy: 0.4464 - val_loss: 0.7913 - val_accuracy: 0.6607
Epoch 2/20
112/112 [==============================] - ETA: 0s - loss: 0.8122 - accuracy: 0.71 - 0s 356us/sample - loss: 0.7725 - accuracy: 0.6607 - val_loss: 0.5964 - val_accuracy: 0.6607
Epoch 3/20
112/112 [==============================] - ETA: 0s - loss: 0.5917 - accuracy: 0.68 - 0s 232us/sample - loss: 0.5560 - accuracy: 0.7589 - val_loss: 0.5211 - val_accuracy: 0.6607
Epoch 4/20
112/112 [==============================] - ETA: 0s - loss: 0.4985 - accuracy: 0.68 - 0s 2ms/sample - loss: 0.4679 - accuracy: 0.7857 - val_loss: 0.4291 - val_accuracy: 0.6696
Epoch 5/20
112/112 [==============================] - ETA: 0s - loss: 0.4204 - accuracy: 0.68 - 0s 2ms/sample - loss: 0.4095 - accuracy: 0.7500 - val_loss: 0.3492 - val_accuracy: 0.9643
Epoch 6/20
112/112 [

Train on 112 samples, validate on 112 samples
Epoch 1/20
112/112 [==============================] - ETA: 0s - loss: 1.7066 - accuracy: 0.28 - 1s 5ms/sample - loss: 1.7671 - accuracy: 0.3393 - val_loss: 1.7243 - val_accuracy: 0.3393
Epoch 2/20
112/112 [==============================] - ETA: 0s - loss: 1.5420 - accuracy: 0.43 - 0s 240us/sample - loss: 1.7030 - accuracy: 0.3393 - val_loss: 1.6627 - val_accuracy: 0.3393
Epoch 3/20
112/112 [==============================] - ETA: 0s - loss: 1.6868 - accuracy: 0.34 - 0s 222us/sample - loss: 1.6430 - accuracy: 0.3393 - val_loss: 1.6033 - val_accuracy: 0.3393
Epoch 4/20
112/112 [==============================] - ETA: 0s - loss: 1.8631 - accuracy: 0.31 - 0s 232us/sample - loss: 1.5877 - accuracy: 0.3393 - val_loss: 1.5471 - val_accuracy: 0.3393
Epoch 5/20
112/112 [==============================] - ETA: 0s - loss: 1.6890 - accuracy: 0.28 - 0s 276us/sample - loss: 1.5307 - accuracy: 0.3393 - val_loss: 1.4958 - val_accuracy: 0.3393
Epoch 6/20
112/1

Train on 112 samples, validate on 112 samples
Epoch 1/20
112/112 [==============================] - ETA: 0s - loss: 1.3991 - accuracy: 0.28 - 1s 6ms/sample - loss: 1.6366 - accuracy: 0.3125 - val_loss: 1.1586 - val_accuracy: 0.3750
Epoch 2/20
112/112 [==============================] - ETA: 0s - loss: 1.1023 - accuracy: 0.40 - 0s 2ms/sample - loss: 1.0590 - accuracy: 0.5000 - val_loss: 0.6989 - val_accuracy: 0.7232
Epoch 3/20
112/112 [==============================] - ETA: 0s - loss: 0.6889 - accuracy: 0.78 - 0s 347us/sample - loss: 0.6447 - accuracy: 0.6875 - val_loss: 0.6241 - val_accuracy: 0.6607
Epoch 4/20
112/112 [==============================] - ETA: 0s - loss: 0.5857 - accuracy: 0.71 - 0s 223us/sample - loss: 0.5896 - accuracy: 0.6607 - val_loss: 0.4812 - val_accuracy: 0.7232
Epoch 5/20
112/112 [==============================] - ETA: 0s - loss: 0.4712 - accuracy: 0.71 - 0s 2ms/sample - loss: 0.4560 - accuracy: 0.8214 - val_loss: 0.4220 - val_accuracy: 0.9732
Epoch 6/20
112/112 [

Train on 112 samples, validate on 112 samples
Epoch 1/20
112/112 [==============================] - ETA: 0s - loss: 1.3997 - accuracy: 0.21 - 1s 5ms/sample - loss: 1.2433 - accuracy: 0.3482 - val_loss: 1.0910 - val_accuracy: 0.6518
Epoch 2/20
112/112 [==============================] - ETA: 0s - loss: 1.1056 - accuracy: 0.59 - 0s 240us/sample - loss: 1.0591 - accuracy: 0.5893 - val_loss: 0.9932 - val_accuracy: 0.5893
Epoch 3/20
112/112 [==============================] - ETA: 0s - loss: 1.0015 - accuracy: 0.75 - 0s 222us/sample - loss: 0.9734 - accuracy: 0.5714 - val_loss: 0.9250 - val_accuracy: 0.6339
Epoch 4/20
112/112 [==============================] - ETA: 0s - loss: 0.9332 - accuracy: 0.59 - 0s 2ms/sample - loss: 0.9066 - accuracy: 0.6607 - val_loss: 0.8610 - val_accuracy: 0.6607
Epoch 5/20
112/112 [==============================] - ETA: 0s - loss: 0.8729 - accuracy: 0.62 - 0s 247us/sample - loss: 0.8470 - accuracy: 0.6607 - val_loss: 0.8221 - val_accuracy: 0.6607
Epoch 6/20
112/112

Train on 112 samples, validate on 112 samples
Epoch 1/20
112/112 [==============================] - ETA: 0s - loss: 1.3349 - accuracy: 0.25 - 1s 5ms/sample - loss: 1.1928 - accuracy: 0.3393 - val_loss: 1.1690 - val_accuracy: 0.3393
Epoch 2/20
112/112 [==============================] - ETA: 0s - loss: 1.2737 - accuracy: 0.25 - 0s 254us/sample - loss: 1.1593 - accuracy: 0.3393 - val_loss: 1.1370 - val_accuracy: 0.3393
Epoch 3/20
112/112 [==============================] - ETA: 0s - loss: 1.2012 - accuracy: 0.31 - 0s 258us/sample - loss: 1.1276 - accuracy: 0.3393 - val_loss: 1.1102 - val_accuracy: 0.3393
Epoch 4/20
112/112 [==============================] - ETA: 0s - loss: 1.1420 - accuracy: 0.31 - 0s 238us/sample - loss: 1.1026 - accuracy: 0.3393 - val_loss: 1.0875 - val_accuracy: 0.3393
Epoch 5/20
112/112 [==============================] - ETA: 0s - loss: 0.9758 - accuracy: 0.43 - 0s 232us/sample - loss: 1.0793 - accuracy: 0.3393 - val_loss: 1.0690 - val_accuracy: 0.3393
Epoch 6/20
112/1

INFO:tensorflow:Oracle triggered exit


In [ ]:
# tuner.search_space_summary()

In [18]:
models = tuner.get_best_models(num_models=2)
print(models[0].summary(),'\n\n',models[1].summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                300       
_________________________________________________________________
dense_1 (Dense)              (None, 60)                3660      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 183       
Total params: 4,143
Trainable params: 4,143
Non-trainable params: 0
_________________________________________________________________
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                400       
_________________________________________________________________
dense_1 (Dense)              (None, 80)                6480      
__________________________________

In [19]:
print(models[0].evaluate(x_test, y_test))
print(models[1].evaluate(x_test, y_test))

38/38 [==============================] - ETA: 0s - loss: 0.0177 - accuracy: 1.00 - 0s 9ms/sample - loss: 0.0322 - accuracy: 1.0000
[0.032227140116064174, 1.0]
38/38 [==============================] - ETA: 0s - loss: 0.0209 - accuracy: 1.00 - 0s 3ms/sample - loss: 0.0358 - accuracy: 1.0000
[0.03578644402717289, 1.0]
